# Eigenfaces demo
<img src="https://lh3.googleusercontent.com/QrehAdN13wxBJrpttgnD2lQMFJh4fmK0C8UQg8R_-j76EBbfYM-Ie3X72VsG2u8s4mg=w300" width="300"/>

## PCA example
This is directly from [the sklearn documentation](http://scikit-learn.org/stable/auto_examples/applications/face_recognition.html)

In [1]:
from time import time
import logging
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC

## download the data

In [2]:
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

## load data into numpy arrays

In [3]:

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7


## Split into a training set and a test set using a stratified k fold

In [5]:
# split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

## Compute a PCA (eigenfaces)
Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled dataset): unsupervised feature extraction / dimensionality reduction

In [6]:
n_components = 150

print("Extracting the top %d eigenfaces from %d faces"
      % (n_components, X_train.shape[0]))
t0 = time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

eigenfaces = pca.components_.reshape((n_components, h, w))

print("Projecting the input data on the eigenfaces orthonormal basis")
t0 = time()
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print("done in %0.3fs" % (time() - t0))

Extracting the top 150 eigenfaces from 966 faces
done in 0.257s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.017s


## Train a SVM classification model

In [11]:
print("Fitting the classifier to the training set")
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
%time clf = clf.fit(X_train_pca, y_train)



Fitting the classifier to the training set
CPU times: user 18.2 s, sys: 18.1 ms, total: 18.2 s
Wall time: 18.2 s


## Quantitative evaluation of the model quality on the test set

In [12]:
print("Predicting people's names on the test set")
t0 = time()
y_pred = clf.predict(X_test_pca)
print("done in %0.3fs" % (time() - t0))

print(classification_report(y_test, y_pred, target_names=target_names))
print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))

Predicting people's names on the test set
done in 0.048s
                   precision    recall  f1-score   support

     Ariel Sharon       0.75      0.46      0.57        13
     Colin Powell       0.81      0.87      0.84        60
  Donald Rumsfeld       0.89      0.63      0.74        27
    George W Bush       0.82      0.98      0.89       146
Gerhard Schroeder       0.95      0.76      0.84        25
      Hugo Chavez       1.00      0.47      0.64        15
       Tony Blair       0.97      0.81      0.88        36

      avg / total       0.86      0.85      0.84       322

[[  6   2   0   5   0   0   0]
 [  1  52   1   6   0   0   0]
 [  1   2  17   7   0   0   0]
 [  0   3   0 143   0   0   0]
 [  0   1   0   4  19   0   1]
 [  0   3   0   4   1   7   0]
 [  0   1   1   5   0   0  29]]


## Qualitative evaluation of the predictions using matplotlib

In [14]:
def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    """Helper function to plot a gallery of portraits"""
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())


# plot the result of the prediction on a portion of the test set

def title(y_pred, y_test, target_names, i):
    pred_name = target_names[y_pred[i]].rsplit(' ', 1)[-1]
    true_name = target_names[y_test[i]].rsplit(' ', 1)[-1]
    return 'predicted: %s\ntrue:      %s' % (pred_name, true_name)

prediction_titles = [title(y_pred, y_test, target_names, i)
                     for i in range(y_pred.shape[0])]

plot_gallery(X_test, prediction_titles, h, w)

# plot the gallery of the most significative eigenfaces

eigenface_titles = ["eigenface %d" % i for i in range(eigenfaces.shape[0])]
plot_gallery(eigenfaces, eigenface_titles, h, w)

plt.show()

<h1 style="color:red">Mini-mini Project</h1>
<h3 style="color:red">75xp</h3>
### 1. variance
Check out the sklearn documentation on PCA paying particular attention to the *Attributes* section. How much (%) of the variance is explained by the first component and the second? Your output should look like:

    array([ 0.22222222,  0.1111111])
    
That's not the answer--it's just the format I would like. The pca attribute you will use gives the variance for all components-- I would like you to display only 2.


### 2. How many PCs to Use?
We will evaluate keeping different number of principal components. When we only have 2 labels, accuracy is a good evaluation metric.  When we have more than 2 labels accuracy is not as popular as the F1 score. 

#### a. First, does a higher or lower F1 score indicate a good classifier?


Answer here

#### b.  As you add more principal components as training features do you expect the classifer to get better, worse, or could go either way.

Answer here

#### c. F1 Score vs. No of PCs used
change `n_components` to 10, 15, 25, 50, 100, 250. For each number, show the F1 score. (for 10 the plotting function will break, but you will still see the F1 scores_. 

In [19]:
# Your code here and in the next few cells

In [20]:
# more code

In [21]:
# more code

### 3. Does more PCs always give better performance? 